In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_id = "heegyu/kogpt-j-base"
rank = 8
lora_alpha = 16
max_length = 400

batch_size = 1
lr = 2e-4
num_epochs = 1
device = 'cuda'

#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b", cache_dir="models")
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b", cache_dir="models", device_map="auto", load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="models")
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir="models")


[2023-08-02 13:06:38,278] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
_ = model.eval()
_ = model.to('cuda:0')

In [3]:
from peft import LoraConfig, TaskType, PeftModel
from peft import get_peft_config, get_peft_model


In [4]:
# load peft_config from json format
import json
model = PeftModel.from_pretrained(model, 'models/heegyu_kogpt-j-base_20230801015657/', is_trainable=True)



In [5]:
import torch


In [6]:
prompt = '''마지막으로 사요를 본 지 몇 년이 지났다. 처음에는 사요가 단지 너무 바쁠 뿐이라고 답하던 양친도, 그런 거짓말을 믿기에는 너무 커버린 히나가 사요에 대해 캐물으면 대놓고 말을 돌렸다.'''


In [7]:
with torch.no_grad():
    tokens = tokenizer(prompt, return_tensors='pt').to(device='cuda')
    gen_tokens = model.generate(
        input_ids=tokens['input_ids'],
        attention_mask=tokens['attention_mask'],
        do_sample=True,
        temperature=0.9,
        max_length=100,
        top_k=10)
    generated = tokenizer.batch_decode(gen_tokens)


In [8]:
print(generated[0])


마지막으로 사요를 본 지 몇 년이 지났다. 처음에는 사요가 단지 너무 바쁠 뿐이라고 답하던 양친도, 그런 거짓말을 믿기에는 너무 커버린 히나가 사요에 대해 캐물으면 대놓고 말을 돌렸다.
지푸라기라도 잡고 싶었던
지푸라기라도 잡고싶어
하지만 그 순간 지푸라기에서 나온 무언가가
어두운 밤길을 걷는
사소한 것들에 대한 집착이
아무것도 남지 않았어
그것은 바로
다
사라져 버린


 # Fine-tune

 ## Parameter-Efficient Fine-tuning

In [9]:
model.train()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTJForCausalLM(
      (transformer): GPTJModel(
        (wte): Embedding(51200, 768)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPTJBlock(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPTJAttention(
              (attn_dropout): Dropout(p=0.0, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=768, out_features=768, bias=False)
              (v_proj): Linear(
                in_features=768, out_features=768, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8

In [10]:
model.print_trainable_parameters()


trainable params: 294,912 || all params: 163,990,016 || trainable%: 0.179835338268398


 ## Preparing Training Dataset

In [47]:
from glob import glob
text_files = glob('./data/*.txt')
text_files[:10]


['./data/제로야슈_아침이 올 때까지.txt',
 './data/치요케이치요_연습.txt',
 './data/200822 인어사요.txt',
 './data/너의 날 선 상냥함으로 외전.txt',
 './data/사요히나 생일.txt',
 './data/히나츠구_한낮의 기다림.txt',
 './data/200312 사요린코사요.txt',
 './data/사요히나사요_수위190216.txt',
 './data/200816 교류회 원고.txt',
 './data/사요히나_너의 날선 상냥함으로.txt']

In [12]:
print(len(text_files))


21


In [48]:
from datasets import load_dataset
dataset = load_dataset('text', data_files=text_files, sample_by='document', split='train')


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /home/smg/.cache/huggingface/datasets/text/default-611af0c7e5fe9a7a/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


In [49]:
dataset


Dataset({
    features: ['text'],
    num_rows: 20
})

In [15]:
tokenizer


GPT2TokenizerFast(name_or_path='heegyu/kogpt-j-base', vocab_size=51200, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)

In [50]:

import re
def preprocess_function(examples):
    batch_size = len(examples["text"])
    # Remove redundant newlines
    for i in range(batch_size):
        examples["text"] = re.sub(r"\n\s*\n", "\n", examples["text"])
    model_inputs = tokenizer(examples["text"], padding='longest', return_tensors="pt")
    return model_inputs

processed_datasets = dataset.map(
    preprocess_function,
    num_proc=1,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
    remove_columns='text'
)

print(processed_datasets)


Running tokenizer on dataset:   0%|          | 0/20 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 20
})


In [53]:
list(processed_datasets[0])

['input_ids', 'attention_mask']

In [59]:
import numpy as np
def chunk_text(examples):
    # Chunk into max_length tokens, return as a list of examples
    result = {}
    for k, v in examples.items():
        value_chunks = []
        v = sum(v, [])
        for v2 in v:
            print(len(v2))
            for i in range(0, len(v2), max_length):
                value_chunks.append(v2[i:i+max_length])
            # Another examples, starting from half of max_length
            for i in range(max_length//2, len(v2), max_length):
                value_chunks.append(v2[i:i+max_length])
        value_chunks = [x for x in value_chunks if len(x) == max_length]
        result[k] = value_chunks[:]
        result['labels'] = result['input_ids'].copy()
    print(len(result['input_ids']))
    print(np.array(result['input_ids']).shape)
    return result

chunked_dataset = processed_datasets.map(
    chunk_text,
    batched=True,
    num_proc=1,
    load_from_cache_file=False,
    desc="Chunking text",
)
print(chunked_dataset)
print(max([len(x) for x in chunked_dataset['input_ids']]))
#print('chunked_dataset[0]:', chunked_dataset[0])

from tqdm import tqdm

assert np.array(chunked_dataset['input_ids']).shape == np.array(chunked_dataset['labels']).shape
assert np.array(chunked_dataset['input_ids']).shape == np.array(chunked_dataset['attention_mask']).shape



Chunking text:   0%|          | 0/20 [00:00<?, ? examples/s]

7871
2909
3317
5544
4155
3789
1522
2724
3437
46784
5784
3517
4386
1643
6635
1315
884
2398
3084
1943
7871
2909
3317
5544
4155
3789
1522
2724
3437
46784
5784
3517
4386
1643
6635
1315
884
2398
3084
1943
536
(536, 400)
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 536
})
400


In [57]:
_ = model.eval()

In [63]:
with torch.no_grad():
    for i in range(100):
        text = tokenizer.decode(chunked_dataset[i]['input_ids'])[:100]
        output = model(**{k: torch.LongTensor(v).to('cuda') for k, v in chunked_dataset[i].items()})
        if output.loss.detach().cpu() > 4:
            print(i)
            print(text)
            print(output.loss)

0

아침이 올 때까지
파이널판타지14
제로 × 야슈톨라 룰
※ 파이널판타지14 “효월의 종언” 6.2 버전까지의 스포일러를 포함하고 있습니다. 6.2 이후 스토리와는 설정충돌이 있을
tensor(4.5662, device='cuda:0')
19
의 하늘 대신 그를 맞이하는 것은 시원한 바람과 푸른 하늘이다. 목 언저리까지 곧게 뻗은 머리카락이 바람에 비단이 물결치듯 일렁인다. 무심코 모자를 내리누르자, 두꺼운 갑주에서 덜
tensor(4.0544, device='cuda:0')
25
둘 걸 그랬어요. 야슈톨라가 중얼거리는 소리가 들려왔다.
습기로 가득한 대기와 끝없이 이어진 지평선은, 라자한의 풍경에 하늘 끝까지 불타오르는 듯한 노을을 만들어 낸다. 에테르를 
tensor(4.0206, device='cuda:0')
48
마음이 반쯤 섞인 목소리로 되물었다. 이제 팔짱은 기본이 된 건지, 케이 옆에 딱 붙은 채였다. 케이의 어깨를 치요코의 머리카락이 간질였다. 케이는 치요코를 돌아보았다.
"별로야?
tensor(4.0190, device='cuda:0')
52
. 어린 시절 사요와 온 성 안을 뛰어다닌, 정확히는 앞서가는 히나를 사요가 쫓아다닌 덕분이었다. 그 중 가장 유용했던 것은, 이젠 말라버린 호수정원 바닥에 있는 수로였다. 이 수
tensor(4.0390, device='cuda:0')
53
일까? 바다생물일까? 강에서 나왔나? 저 멀리 밀림에는 강에 사는 사람만한 동물이 있다고 들었다. 여기에도 그런 동물이 있는 걸까?
물이 끓어오르듯 한번에 기화점에 도달한 히나의 
tensor(4.1125, device='cuda:0')
57
 바다생물일지도 몰라.
어쩌면 인어일지도.
그래. 바닷속에도 왕국이 있는 거야. 그리고 그곳의 공녀가 저녁노을이 보고 싶어서 여기로 오는 거야. 내가 그러는 것처럼.
가슴이 뛰었다
tensor(4.1597, device='cuda:0')
58
 신비한 괴물들.
인어
히나가 눈을 크게 떴다.
사람의 아이로 태어나, 자라면 정체

In [64]:
print(tokenizer.decode(chunked_dataset[19]['input_ids']))

의 하늘 대신 그를 맞이하는 것은 시원한 바람과 푸른 하늘이다. 목 언저리까지 곧게 뻗은 머리카락이 바람에 비단이 물결치듯 일렁인다. 무심코 모자를 내리누르자, 두꺼운 갑주에서 덜거덕 소리가 난다.
아마 이곳은 나의 고향과 가장 거리가 먼 곳이리라고 제로는 생각했다. 오직 허무만이 뭇 생명을 감싸고 있는 세계. 약육강식의 세계. 죽는 것마저 허락되지 않는 그곳에서, 공기에 깃든 경고를 미리 읽어내는 감각은 의미가 없었다.
지금은 어디를 가나 공기가 메시지를 전하고 있다. 코를 자극하는 후추의 향이 제로에게는 그다지 불쾌하지 않았다. 후추, 아마 후추가 맞을 것이다, 라고 제로는 생각했다. 제로는 사라져 버린 감각들을 다시 제대로 이용하는 법을 배우는 중이었다. 새로운 생존방식에 익숙해지기 위해서였다. 특히, 엄청나게 매운 카레를 주는 대로 입에 넣었다가 모두의 앞에서 그대로 뱉어버린 뒤부터는, 다들 먹을 수 있을지 없을지 구별하는 법 정도는 배우라며 성화였다. 제로는 그런 요란스러움이 익숙하지 않았다. 하지만 싫지만도 않다고 생각했다.
“어머, 그렇게 보다간 하늘에 구멍이 뚫리겠어요.”
회상에 잠기던 제로에게 진담을 하는 건지 아닌지 모를 목소리가 들려왔다. 뒤를 돌아보자 그 익숙한 목소리의 주인이 팔짱을 낀 채 서 있었다. 한참 전부터 보고 있었을지도 모른다. 제로는 그렇게 생각하며 야슈톨라에게 되물었다.
“……하늘에 구멍이 날 수가 있나?”
순간 야슈톨라의 귀가 조금 쫑긋이는 듯 했다. 야슈톨라가 답했다.
“후후, 농담이에요. 아직 유머감각은 익히지 않았나 봐요?”
제로는 한숨을 내쉬었다.
“……유한한 삶을 사는 인간이 그런 농담에 시간을 쓰러 여기까지 오다니, 정말 한가로운가 보군.”
“어머, 비꼴 줄은 아는군요?”
야슈톨라는 눈썹을 치켜세우며 한 치도 물러서지 않고 되받아쳤다. 그러다 조금 표정을


In [62]:
print(tokenizer.decode(chunked_dataset[20]['input_ids']))

 누그러뜨리고 말을 이었다.
“미안하지만 한가롭진 않았어요. 그런데 저랑 같이 바쁠 예정이던 어느 분이 못 나타나게 되었거든요.”
“……그걸 내가 알아야 하는 이유가 있나?”
“한가로운지 아닌지 먼저 물어본 건 당신 아니었나요?”
야슈톨라는 다시 되받아치다가, 어떤 반발심도 없이 무표정한 제로의 얼굴을 보고 얕게 한숨을 쉬었다.
“미안해요. 가끔…… 당신은 이유가 없으면 행동하지 않는 생활방식에 익숙해져 있다는 걸 잊어버리게 되네요.”
“미안해 할 필요는 없다만.”
제로는 그렇게 말하며 쓰고 있던 모자를 내리눌렀다. 잠깐의 침묵이 흘렀다. 자신이 한 말이 야슈톨라의 신경을 거슬렀다는 걸 제로도 아예 모르는 바는 아니었지만, 그렇다고 해서 적극적으로 기분을 풀어줄 마음도 없었다. 침묵을 지키고 있자, 눌러쓴 모자의 챙 너머에서 가벼운 박수 소리가 났다.
“그럼 이건 어떨까요? 당신이 그 사람 대신에 시간 좀 내 줘요. 가벼운 데이트라고 할까요.”
고개를 든 제로의 눈에 들어오는 건 뜻 모를 표정을 짓는 야슈톨라였다. 그 표정은 기분이 좋은 듯도, 놀리는 듯도 했다.
“내가 거기에 응해서 얻는 게 있나?”
“당신도 이 세계에서의 생활에 익숙해져야 하지 않겠어요? 언제까지나 이방인처럼 고독하게 살 수는 없잖아요.”
“별로 상관은 없다만…….”
야슈톨라는 그 말을 한숨으로 받았다. 이번에야말로 기분이 꽤 상한 건지, 발로 탁탁 차며 귀를 뒤로 젖혔다. 야슈톨라를 잘 아는 사람이라면 세계를 구한 영웅이라도 무심코 뒤로 물러나게 할 만큼 강한 분노의 표현이었다. 그러나 그것도 제로에게는 큰 효과가 없었다. 이윽고 야슈톨라가 포기했다는 듯 과장된 동작으로 양손을 들었다.
“알았어요. 정말 손이 많이 가는군요. 그럼 이건 어때요? 오늘 하루 동안 가는대로 따라 와주면 에테르를 보충하게 해주겠어요.”
제로는 눈을 치켜떴다. 그리고


In [24]:
torch.cuda.empty_cache()


In [ ]:
import bitsandbytes as bnb


In [ ]:
import transformers


In [ ]:
training_args=transformers.TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        num_train_epochs=num_epochs,
        #max_steps=200,
        learning_rate=lr, 
        fp16=True,
        logging_steps=10, 
        optim='adafactor',
        output_dir='outputs'
    )

In [ ]:
""" from accelerate import Accelerator
accelerator = Accelerator(mixed_precision='fp16')

model.to(accelerator.device)
 """

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=chunked_dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)


In [ ]:
with torch.cuda.amp.autocast(): 
    trainer.train()


In [ ]:
model.eval()


In [ ]:
with torch.no_grad():
    tokens = tokenizer(prompt, return_tensors='pt').to(device='cuda')
    gen_tokens = model.generate(
        input_ids=tokens['input_ids'],
        attention_mask=tokens['attention_mask'],
        do_sample=True,
        temperature=0.9,
        max_length=100,
        top_k=10,
        repetition_penalty=10.0)
    generated = tokenizer.batch_decode(gen_tokens)
    print(generated[0])


In [ ]:
# Save LoRA model with timestamp
import datetime
model_id = "heegyu/kogpt-j-base"
model.save_pretrained(f"models/{model_id.replace('/', '_')}_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}")
